In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

In [2]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

In [3]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})

In [4]:
df_cas

,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,LIMA,PR,25.0,MASCULINO,20201217.0
1,ICA,PR,20.0,FEMENINO,20200822.0
2,HUANUCO,PR,22.0,FEMENINO,20200729.0
3,ANCASH,AG,18.0,FEMENINO,20210630.0
4,ANCASH,AG,17.0,MASCULINO,20210404.0
...,...,...,...,...,...
2562530,LIMA,PCR,23.0,MASCULINO,20200423.0
2562531,LIMA,PCR,47.0,FEMENINO,20210122.0
2562532,CALLAO,PCR,52.0,MASCULINO,20210911.0
2562533,LIMA,PCR,50.0,FEMENINO,20200518.0


In [5]:
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32'})

df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('int8')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8') 

# Transformar a formato fecha
df_cas.loc[:,'fecha_resultado'] = pd.to_datetime(df_cas['fecha_resultado'], format = '%Y%m%d')

# Agregar la diferencia en días desde el primer caso
fecha_min = df_cas['fecha_resultado'].min()

def days_between(d1, d2):
    return abs((d2 - d1).days)

df_cas['time_days'] = df_cas['fecha_resultado'].apply(lambda row : days_between(row, fecha_min)+1)

# Agregar la semana epidemiológica y el año
df_cas[['year','epi_week']] = df_cas['fecha_resultado'].apply(lambda date : Week.fromdate(date) \
                                                                                  .weektuple()) \
                                                                                  .tolist()
# Reorder
df_cas = df_cas[['fecha_resultado','year', 'epi_week', 'time_days', 
                  'sexo', 'edad', 'edad_cat', 'departamento', 'metododx']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

In [6]:
df_cas.to_csv('../Data/DP5_covid19-peru_info-casos-positivos.csv')